<a href="https://colab.research.google.com/github/PrathameshGawasbuild/Prathamesh/blob/main/ASSIGNMENT_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:

# Prathamesh Gawas 23CV308
# Concrete Mix Design Calculation as per IS 10262:2019 and IS 456:2000

# Input Section
fck = float(input("Enter the value of characteristic compressive strength (fck): "))

# Experimental Determinations
Gca = float(input("Enter the value of specific gravity of Coarse Aggregate (CA): "))
Gfa = float(input("Enter the value of specific gravity of Fine Aggregate (FA): "))
Gc = float(input("Enter the value of specific gravity of Cement: "))
Water_Density = float(input("Enter the value of Water Density (kg/m³): "))
AGG_Size = int(input("Enter the nominal size of aggregate (10/20/40 mm): "))
Nature_of_AGG = input("Nature of Aggregates (Angular/Sub-Angular/Gravel/Round): ").strip().title()
Slump = float(input("Enter the value of workability (Slump in mm): "))
Admixture = input("Type of Admixture (None/Plastisizer/Super-plastisizer): ").strip().title()
Exposure_Condition = input("Exposure Condition (Mild/Moderate/Severe/Very Severe/Extreme): ").strip().title()
Concreting = input("Type of Concreting (Plain/Reinforced): ").strip().lower()
Zone = int(input("Enter Zone (1/2/3/4): "))

# Target Mean Strength (IS 10262:2019)
sigma = {
    10: 3.5, 15: 3.5, 20: 4, 25: 4, 30: 5, 35: 5, 40: 5, 45: 5, 50: 5, 55: 5
}

if fck not in sigma:
    print("Warning: fck value not in standard table, using nearest available value.")
    fck = min(sigma.keys(), key=lambda x: abs(x - fck))

ft = fck + 1.65 * sigma[fck]
print("\nTarget Mean Strength:", ft, "MPa")

# Maximum free Water-Cement Ratio (IS 456:2000 Table 5)
if Concreting == "plain":
    WC_ratio_table = {
        "Mild": 0.6,
        "Moderate": 0.6,
        "Severe": 0.5,
        "Very Severe": 0.45,
        "Extreme": 0.4
    }
else:
    WC_ratio_table = {
        "Mild": 0.55,
        "Moderate": 0.5,
        "Severe": 0.45,
        "Very Severe": 0.45,
        "Extreme": 0.4
    }

WC_ratio = WC_ratio_table[Exposure_Condition]
print("Water-Cement Ratio:", WC_ratio)

# Minimum Cement Content (IS 456:2000 Table 5)
if Concreting == "plain":
    Min_Cement_Content = {
        "Mild": 220,
        "Moderate": 240,
        "Severe": 250,
        "Very Severe": 260,
        "Extreme": 280
    }
else:
    Min_Cement_Content = {
        "Mild": 300,
        "Moderate": 300,
        "Severe": 320,
        "Very Severe": 340,
        "Extreme": 360
    }

print("Minimum Cement Content:", Min_Cement_Content[Exposure_Condition], "kg/m³")

# Water Content (IS 10262:2019 Table 2)
Water_Content_Table = {
    10: 208,
    20: 186,
    40: 165
}

Water_Content = Water_Content_Table.get(AGG_Size, 186)  # Default to 20mm

# Adjustment for slump
if Slump > 50:
    Water_Content += Water_Content * (0.03 * ((Slump - 50) // 25))

# Adjustment for aggregate shape
if Nature_of_AGG == "Sub-Angular":
    Water_Content -= 10
elif Nature_of_AGG == "Gravel":
    Water_Content -= 20
elif Nature_of_AGG == "Round":
    Water_Content -= 25

# Adjustment for admixture
if Admixture == "Plastisizer":
    Water_Content -= 0.1 * Water_Content
elif Admixture == "Super-Plastisizer":
    Water_Content -= 0.2 * Water_Content

print("Water Content:", round(Water_Content, 2), "kg/m³")

# Cement Content
Cement_Content = Water_Content / WC_ratio
if Cement_Content > 450:
    print("Cement Content exceeds 450 kg/m³ (limit as per IS 456:2000). Using 450 kg/m³.")
    Cement_Content = 450
else:
    print("Cement Content:", round(Cement_Content, 2), "kg/m³")

# Volume Calculations
Vol_Cement = Cement_Content / (Gc * Water_Density)
Vol_Water = Water_Content / Water_Density
Vol_AGG = 1 - (Vol_Water + Vol_Cement)

print("\nVolume of Cement:", round(Vol_Cement, 4), "m³")
print("Volume of Water:", round(Vol_Water, 4), "m³")
print("Volume of Aggregates (CA + FA):", round(Vol_AGG, 4), "m³")

# Aggregate Fraction based on Zone (IS 10262:2019 Table 3)
Zone_ID = {
    1: {10: 0.44, 20: 0.60, 40: 0.69},
    2: {10: 0.46, 20: 0.62, 40: 0.71},
    3: {10: 0.48, 20: 0.64, 40: 0.73},
    4: {10: 0.50, 20: 0.66, 40: 0.75}
}

Fraction = Zone_ID[Zone][AGG_Size]

# Adjust fraction based on W/C ratio
if WC_ratio == 0.45:
    Fraction += 0.01 * Fraction
elif WC_ratio == 0.40:
    Fraction += 0.02 * Fraction
elif WC_ratio == 0.55:
    Fraction -= 0.01 * Fraction
elif WC_ratio == 0.60:
    Fraction -= 0.02 * Fraction

print("Coarse Aggregate Fraction:", round(Fraction, 3))

# Volumes and Masses of Aggregates
Vol_CA = Vol_AGG * Fraction
Vol_FA = Vol_AGG - Vol_CA
Mass_CA = Vol_CA * Gca * Water_Density
Mass_FA = Vol_FA * Gfa * Water_Density

print("Volume of Coarse Aggregate:", round(Vol_CA, 4), "m³")
print("Volume of Fine Aggregate:", round(Vol_FA, 4), "m³")
print("Mass of Coarse Aggregate:", round(Mass_CA, 2), "kg/m³")
print("Mass of Fine Aggregate:", round(Mass_FA, 2), "kg/m³")

# Ratios
print("\n--- Mix Proportion (By Weight) ---")
print(f"1 : {round(Mass_FA / Cement_Content, 2)} : {round(Mass_CA / Cement_Content, 2)} | W/C = {round(WC_ratio, 2)}")

print("\n--- Mix Proportion (By Volume) ---")
print(f"1 : {round(Vol_FA / Vol_Cement, 2)} : {round(Vol_CA / Vol_Cement, 2)} : {round(Vol_Water / Vol_Cement, 2)}")


Enter the value of characteristic compressive strength (fck): 40
Enter the value of specific gravity of Coarse Aggregate (CA): 2.74
Enter the value of specific gravity of Fine Aggregate (FA): 2.74
Enter the value of specific gravity of Cement: 3.15
Enter the value of Water Density (kg/m³): 1000
Enter the nominal size of aggregate (10/20/40 mm): 20
Nature of Aggregates (Angular/Sub-Angular/Gravel/Round): sub-angular
Enter the value of workability (Slump in mm): 100
Type of Admixture (None/Plastisizer/Super-plastisizer): super-plastisizer
Exposure Condition (Mild/Moderate/Severe/Very Severe/Extreme): severe
Type of Concreting (Plain/Reinforced): reinforced
Enter Zone (1/2/3/4): 1

Target Mean Strength: 48.25 MPa
Water-Cement Ratio: 0.45
Minimum Cement Content: 320 kg/m³
Water Content: 149.73 kg/m³
Cement Content: 332.73 kg/m³

Volume of Cement: 0.1056 m³
Volume of Water: 0.1497 m³
Volume of Aggregates (CA + FA): 0.7446 m³
Coarse Aggregate Fraction: 0.606
Volume of Coarse Aggregate: 0.451

Enter the value of design speed: 65
Enter the value of Radius of curvature: 220
Enter the value of slope: 150
Enter the value of width of road including extra widening: 7.5
'enter the value for plain terain:0.07
The value of Super elevation: 0.08535353535353535
0.07
The length of transition curve: 39.37500000000001
 Constant R: 48
 Constant C: 16
Total Data Values for EWL Constant: 4
Total Data Values for AADT: 4
Enter EWL Constant:
330
Enter EWL Constant:
1070
Enter EWL Constant:
2460
Enter EWL Constant:
4620
Enter AADT: 
3750
Enter AADT: 
470
Enter AADT: 
320
Enter AADT: 
120
 Total EWL : 3082000.0
EWL after 60 years : 4931200.0
Traffic Index :  7.577910657490486
Pavement Thickness:  36.847136933326986 cm
 Load in kg: 4085
 Tyre pressure kg/cm^2: 7
Total Number of layers in a given Pavement : 3
California Bearing Ratio of Material in %
6
Thickness Above this layer:  31.712799015896838 cm
California Bearing Ratio of Material in %
6
Thickness Above this layer:  31.712799015896838 cm
Ca